In [66]:
from PIL import Image 
from os import listdir 
from os.path import isdir, join, isfile
from numpy import asarray, expand_dims
import numpy as np

In [67]:
def load_face(filename):
    try:
        # Carregar a imagem
        image = Image.open(filename).convert("RGB")
        return asarray(image)
    except Exception as e:
        print(f"Erro ao carregar a imagem {filename}: {e}")
        return None

### Carregando imagens de face de um diretório

In [68]:
def load_faces(directory_src):
    faces = []
    print(f"Verificando o diretório: {directory_src}")
    for filename in listdir(directory_src):
        path = join(directory_src, filename)

        # Verificar se é um arquivo de imagem válido
        if not isfile(path) or not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Ignorado (não é imagem): {filename}")
            continue

        face = load_face(path)
        if face is not None:
            faces.append(face)
    print(f"Faces carregadas no diretório {directory_src}: {len(faces)}")
    return faces

### Carregando todo o dataset de imagens de faces

In [69]:
def load_fotos(directory_src):
    X, y = [], []
    print(f"Carregando imagens de: {directory_src}")
    for subdir in listdir(directory_src):
        path = join(directory_src, subdir, '')

        # Verificar se é um diretório
        if not isdir(path):
            print(f"Ignorado (não é diretório): {subdir}")
            continue

        print(f"Processando subdiretório: {subdir}")
        faces = load_faces(path)
        labels = [subdir for _ in range(len(faces))]
        print(f"Subdiretório {subdir}: {len(faces)} faces carregadas")
        X.extend(faces)
        y.extend(labels)
    print(f"Total de imagens: {len(X)}, Total de rótulos: {len(y)}")
    return asarray(X), asarray(y)

### Carregando todas as imagens

In [70]:
trainX, trainy = load_fotos("C:\\datasets\\bio-facial\\train")

Carregando imagens de: C:\datasets\bio-facial\train
Processando subdiretório: anaF
Verificando o diretório: C:\datasets\bio-facial\train\anaF\
Faces carregadas no diretório C:\datasets\bio-facial\train\anaF\: 94
Subdiretório anaF: 94 faces carregadas
Processando subdiretório: wandersonF
Verificando o diretório: C:\datasets\bio-facial\train\wandersonF\
Faces carregadas no diretório C:\datasets\bio-facial\train\wandersonF\: 98
Subdiretório wandersonF: 98 faces carregadas
Total de imagens: 192, Total de rótulos: 192


In [71]:
trainX.shape

(192, 160, 160, 3)

In [72]:
trainy.shape

(192,)

## Usando face-net

In [73]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


model = load_model('facenet_keras.h5')

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
print("Modelo carregado com sucesso!")


Modelo carregado com sucesso!


### Função geradora de embedding

In [74]:
def get_embedding(model, face_pixels):
    #Padronização
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std

    #Transformar a face em um unico exemplo
    ## (160, 160) -> (1, 160, 160)

    samples = expand_dims(face_pixels, axis=0)

    # Realizar a predição gerando o embedding
    yhat = model.predict(samples)

    return yhat[0]

In [75]:
newTrainX = list()

In [76]:
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)

In [77]:
#Convertendo em numpy porque é mais rápido.
newTrainX = asarray(newTrainX)

In [78]:
newTrainX.shape

(192, 128)

### Transformando o modelo em um dataset

In [80]:
import pandas as pd

In [81]:
df = pd.DataFrame(data=newTrainX)

In [82]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.289921,-1.152140,0.239283,-0.929706,-1.263536,0.448747,0.537707,0.462252,0.574043,0.559753,...,-0.734085,-0.451969,1.067420,0.492537,-1.641094,-0.775325,-1.663003,-0.138243,-0.513796,-1.420127
1,-1.203166,-0.103692,0.850324,-1.978575,-0.281943,-0.283227,-0.108825,1.448760,-1.487779,-1.186183,...,-2.016483,-0.678227,1.523620,0.761791,1.105002,-0.334576,-1.335838,-0.160857,-1.163937,-0.298558
2,-0.602753,-0.266089,1.301060,-1.102458,-1.344989,-0.199267,0.790890,0.224404,-1.976963,0.638057,...,-1.139645,0.696575,1.113710,0.298694,-0.139617,-0.568771,-0.964362,1.508376,0.941980,-0.976072
3,-0.888109,-1.216653,0.396816,-1.204848,0.227774,-0.364578,0.781512,0.513624,-1.917076,0.767922,...,-2.335488,-0.619197,0.819682,1.081258,1.064052,-0.498247,-1.187200,0.242233,-0.071040,-1.339557
4,-0.798776,-0.394654,1.078107,-0.997775,-1.114724,-0.409730,0.948913,0.286215,-2.238382,0.719750,...,-1.192232,0.801373,1.063006,0.136837,-0.289999,-0.486712,-1.222666,1.741000,1.285398,-0.972803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.992916,-0.022542,0.036196,1.745325,0.854401,2.047154,1.957049,0.617729,0.811754,-0.294685,...,0.179397,-0.877500,1.175461,-1.358159,0.873499,-1.008092,-0.193851,-0.866488,-0.916496,-0.217651
188,-1.582244,-0.128047,0.273325,1.887661,0.156800,2.048656,2.426804,0.860428,1.191629,-0.455625,...,0.294762,-0.526193,-0.852951,0.062449,0.926860,-0.522391,-0.416123,0.176782,-0.447783,-0.256754
189,-1.024907,0.118502,0.242778,1.743642,0.363596,2.089461,2.009594,1.129666,1.391434,-0.725070,...,-0.291158,-0.689345,-0.470598,-0.082423,1.433930,-0.518543,-0.072709,-0.202007,-0.169454,-0.211468
190,-0.935544,-0.088811,-0.221225,1.421503,0.625742,2.130554,1.747446,1.641021,1.831261,-1.065332,...,-0.193376,-1.340125,-1.258122,0.036564,1.518143,-0.260979,0.107894,0.251887,-0.183203,0.405607


### Acrescentando o rotulo dos embeddings

In [83]:
df['target'] = trainy

In [84]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-0.289921,-1.152140,0.239283,-0.929706,-1.263536,0.448747,0.537707,0.462252,0.574043,0.559753,...,-0.451969,1.067420,0.492537,-1.641094,-0.775325,-1.663003,-0.138243,-0.513796,-1.420127,anaF
1,-1.203166,-0.103692,0.850324,-1.978575,-0.281943,-0.283227,-0.108825,1.448760,-1.487779,-1.186183,...,-0.678227,1.523620,0.761791,1.105002,-0.334576,-1.335838,-0.160857,-1.163937,-0.298558,anaF
2,-0.602753,-0.266089,1.301060,-1.102458,-1.344989,-0.199267,0.790890,0.224404,-1.976963,0.638057,...,0.696575,1.113710,0.298694,-0.139617,-0.568771,-0.964362,1.508376,0.941980,-0.976072,anaF
3,-0.888109,-1.216653,0.396816,-1.204848,0.227774,-0.364578,0.781512,0.513624,-1.917076,0.767922,...,-0.619197,0.819682,1.081258,1.064052,-0.498247,-1.187200,0.242233,-0.071040,-1.339557,anaF
4,-0.798776,-0.394654,1.078107,-0.997775,-1.114724,-0.409730,0.948913,0.286215,-2.238382,0.719750,...,0.801373,1.063006,0.136837,-0.289999,-0.486712,-1.222666,1.741000,1.285398,-0.972803,anaF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.992916,-0.022542,0.036196,1.745325,0.854401,2.047154,1.957049,0.617729,0.811754,-0.294685,...,-0.877500,1.175461,-1.358159,0.873499,-1.008092,-0.193851,-0.866488,-0.916496,-0.217651,wandersonF
188,-1.582244,-0.128047,0.273325,1.887661,0.156800,2.048656,2.426804,0.860428,1.191629,-0.455625,...,-0.526193,-0.852951,0.062449,0.926860,-0.522391,-0.416123,0.176782,-0.447783,-0.256754,wandersonF
189,-1.024907,0.118502,0.242778,1.743642,0.363596,2.089461,2.009594,1.129666,1.391434,-0.725070,...,-0.689345,-0.470598,-0.082423,1.433930,-0.518543,-0.072709,-0.202007,-0.169454,-0.211468,wandersonF
190,-0.935544,-0.088811,-0.221225,1.421503,0.625742,2.130554,1.747446,1.641021,1.831261,-1.065332,...,-1.340125,-1.258122,0.036564,1.518143,-0.260979,0.107894,0.251887,-0.183203,0.405607,wandersonF


### Salvando em arquivo CSV

In [85]:
df.to_csv('faces2.csv')

In [88]:
from sklearn.utils import shuffle

In [89]:
X, y = shuffle(newTrainX, trainy, random_state=0)